In [47]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install pandas matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.5 MB/s eta 0:00:00


In [48]:
import os
path_to_dataset = ""
#Per comodità, per capire se stiamo eseguendo su Google Colab o su Kaggle
if os.path.exists('/content/'):
        path_to_dataset = "/content/train_v2_drcat_02.csv"

    # Verifica se è presente la cartella "input" caratteristica di Kaggle
if os.path.exists('/kaggle/input/'):
        path_to_dataset = "/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv"


In [49]:
import pandas as pd
df = pd.read_csv(path_to_dataset)
#df['text'] = df['text'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44868 entries, 0 to 44867
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           44868 non-null  object
 1   label          44868 non-null  int64 
 2   prompt_name    44868 non-null  object
 3   source         44868 non-null  object
 4   RDizzl3_seven  44868 non-null  bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 1.4+ MB


### Creazione di train e test

In [50]:
df = df.drop('source',axis = 1)
df = df.drop('RDizzl3_seven',axis = 1)
data = df
data = data.drop('label',axis = 1)
data = data.drop('prompt_name',axis = 1)

In [51]:
data

,text
0,Phones\n\nModern humans today are always on th...
1,This essay will explain if drivers should or s...
2,Driving while the use of cellular devices\n\nT...
3,Phones & Driving\n\nDrivers should not be able...
4,Cell Phone Operation While Driving\n\nThe abil...
...,...
44863,"Dear Senator,\n\nI am writing to you today to ..."
44864,"Dear Senator,\n\nI am writing to you today to ..."
44865,"Dear Senator,\n\nI am writing to you today to ..."
44866,"Dear Senator,\n\nI am writing to you today to ..."


In [52]:
lunghezza_massima = max(len(sequenza.split()) for sequenza in df['text'])

print("Lunghezza massima delle sequenze:", lunghezza_massima)

Lunghezza massima delle sequenze: 1656


In [53]:
import math
# Calcola la lunghezza media delle sequenze nel dataset
lunghezza_medio = sum(len(sequenza.split()) for sequenza in df['text']) / len(df['text'])
lunghezza_medio = math.ceil(lunghezza_medio)
print("Lunghezza media delle sequenze:", lunghezza_medio)

Lunghezza media delle sequenze: 384


In [54]:
import numpy as np
# Calcola la lunghezza delle sequenze
lunghezze_sequenze = [len(sequenza.split()) for sequenza in df['text']]

# Scegli il percentile desiderato (ad esempio, il 95° percentile)
percentile_desiderato = 95

# Calcola il valore del percentile
lunghezza_percentile = np.percentile(lunghezze_sequenze, percentile_desiderato)

lunghezza_percentile =  math.ceil(lunghezza_percentile)

print(f"{percentile_desiderato}° percentile della lunghezza delle sequenze:", lunghezza_percentile)

95° percentile della lunghezza delle sequenze: 713


In [55]:
from sklearn.model_selection import train_test_split

# Supponiamo che il tuo dataset sia rappresentato da X (feature) e y (etichette)
X_train, X_temp, y_train, y_temp = train_test_split(data['text'], df['label'], test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

### Create text encoder


In [56]:
import tensorflow as tf

In [57]:
VOCAB_SIZE = 1000 #Parametro da sistemare
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE) #output_sequence_length=50
encoder.adapt(X_train.values.flatten())

In [58]:
#X_train.head() non serve perchè lo fa la rete

In [59]:
#X_train = encoder(X_train) non serve perchè lo fa la rete

In [60]:
#X_train non serve perchè lo fa la rete

##### Controlliamo se l'encoder funziona

In [61]:
import numpy as np
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'to', 'and', 'a', 'of', 'in', 'that', 'is',
       'it', 'for', 'be', 'you', 'can', 'they', 'are', 'have', 'students',
       'i'], dtype='<U15')

### Applichiamo il Padding (VEDERE MEGLIO COME FARE QUESTA COSA)


In [62]:
#from tensorflow.keras.preprocessing.sequence import pad_sequences
#X_train = pad_sequences(X_train, padding='post', maxlen=100)

In [63]:
#X_train

### Creiamo un embedding

In [64]:
embedding_layer = tf.keras.layers.Embedding(1000, 5) #SERVE SOLO DA ESEMPIO IL VERO PADDING è NELLA RETE

In [65]:
#TEST PER VEDERE SE FUNZIONA IL LAYER EMBEDDING #SERVE SOLO DA ESEMPIO IL VERO PADDING è NELLA RETE
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.02189764,  0.01844749,  0.03760279, -0.03621973, -0.04450296],
       [ 0.04972217,  0.0263911 ,  0.01721891,  0.03477538, -0.00472718],
       [ 0.00657231, -0.02893483,  0.02589095, -0.02704761,  0.01037016]],
      dtype=float32)

## Creazione modello

In [66]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        #gestire anche dimensione input e output
        input_dim=len(encoder.get_vocabulary()),
        output_dim=3,
        # aggiungere input_length  =  lunghezza massima o come media o come percentile 95%
        input_length = lunghezza_medio,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [67]:
#model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#              optimizer=tf.keras.optimizers.Adam(1e-4), # default 0.01
#              metrics=['accuracy'])
# -> Da un warning strano, quello di sotto no
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
 800/1122 [====================>.........] - ETA: 3:20 - loss: 0.1943 - accuracy: 0.9182

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score



y_pred = model.predict(X_test)
y_pred = y_pred.astype(int)
y_test = y_test.astype(int)

# Stampa il classification report e l'accuracy score
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(report)
print("****")
print(accuracy)




In [ ]:
print(report)
print("****")
print(accuracy)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
for key in X_test :
    predictions = model.predict(np.array([key]))
    print(predictions)
    break

In [ ]:
for key in y_test :
    print(key)
    break